# Detectron2 Fine-tuning for Electronic Components Detection

This notebook demonstrates fine-tuning a pre-trained Detectron2 model on the electronic components dataset for pre-labeling workflow.

**Goal**: Train a model that can pre-label new images, allowing human labelers to focus on minor corrections rather than full annotation.

## 1. Setup and Imports

In [ ]:
import os
import json
import random
import cv2
import matplotlib.pyplot as plt
from datetime import datetime

# Detectron2 imports
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

print(f"Detectron2 version: {detectron2.__version__}")
print(f"Working directory: {os.getcwd()}")

## 2. Register COCO Dataset

Register the electronic components dataset in COCO format.

In [ ]:
# Define dataset paths
DATA_ROOT = "../dataset"
TRAIN_JSON = os.path.join(DATA_ROOT, "train/_annotations.coco.json")
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
VAL_JSON = os.path.join(DATA_ROOT, "valid/_annotations.coco.json")
VAL_DIR = os.path.join(DATA_ROOT, "valid")
TEST_JSON = os.path.join(DATA_ROOT, "test/_annotations.coco.json")
TEST_DIR = os.path.join(DATA_ROOT, "test")

# Register datasets
register_coco_instances("electronics_train", {}, TRAIN_JSON, TRAIN_DIR)
register_coco_instances("electronics_val", {}, VAL_JSON, VAL_DIR)
register_coco_instances("electronics_test", {}, TEST_JSON, TEST_DIR)

# Get metadata
metadata = MetadataCatalog.get("electronics_train")
dataset_dicts = DatasetCatalog.get("electronics_train")

print(f"Number of training images: {len(dataset_dicts)}")
print(f"Categories: {metadata.thing_classes}")
print(f"Number of categories: {len(metadata.thing_classes)}")

## 3. Visualize Sample Data

Let's visualize a few samples from the training set to verify the data is loaded correctly.

In [ ]:
# Visualize random samples
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, ax in enumerate(axes):
    d = random.choice(dataset_dicts)
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    ax.imshow(out.get_image())
    ax.axis('off')
    ax.set_title(f"Sample {i+1}")

plt.tight_layout()
plt.savefig('../outputs/predictions/training_samples.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Sample visualization saved to outputs/predictions/training_samples.png")

## 4. Configure Model

Using Faster R-CNN with ResNet-50 FPN backbone. This is a good balance between speed and accuracy for quick training.

In [ ]:
cfg = get_cfg()

# Load pre-trained model from model zoo
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

# Dataset configuration
cfg.DATASETS.TRAIN = ("electronics_train",)
cfg.DATASETS.TEST = ("electronics_val",)
cfg.DATALOADER.NUM_WORKERS = 2

# Training configuration - optimized for quick demo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # Learning rate
cfg.SOLVER.MAX_ITER = 500     # Quick training for demo (increase to 2000+ for better results)
cfg.SOLVER.STEPS = (300, 450)  # Learning rate decay steps
cfg.SOLVER.CHECKPOINT_PERIOD = 250  # Save checkpoint every N iterations

# Model configuration
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(metadata.thing_classes)

# Output directory
cfg.OUTPUT_DIR = "../outputs/models"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Use MPS (Metal Performance Shaders) for Mac GPU acceleration if available
# Detectron2 may not support MPS directly, so it will fall back to CPU
# cfg.MODEL.DEVICE = "mps"  # Uncomment if MPS is supported

print("Configuration:")
print(f"  Model: Faster R-CNN R50-FPN")
print(f"  Number of classes: {cfg.MODEL.ROI_HEADS.NUM_CLASSES}")
print(f"  Max iterations: {cfg.SOLVER.MAX_ITER}")
print(f"  Learning rate: {cfg.SOLVER.BASE_LR}")
print(f"  Output directory: {cfg.OUTPUT_DIR}")

## 5. Train the Model

This will run for ~500 iterations. On Mac Studio, this should take 1-2 hours.

**Note**: You can monitor the training progress in the output. Loss values should decrease over time.

In [ ]:
# Create trainer and start training
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

print(f"\n🚀 Starting training at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Training for {cfg.SOLVER.MAX_ITER} iterations...\n")

trainer.train()

print(f"\n✅ Training complete at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Model saved to: {cfg.OUTPUT_DIR}")

## 6. Evaluate on Validation Set

Let's see how well the model performs on unseen validation data.

In [ ]:
# Configure for inference
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set confidence threshold

# Evaluate on validation set
evaluator = COCOEvaluator("electronics_val", output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "electronics_val")
predictor = DefaultPredictor(cfg)

print("Running evaluation on validation set...")
results = inference_on_dataset(predictor.model, val_loader, evaluator)
print("\nEvaluation Results:")
print(results)

## 7. Test Predictions on Sample Images

Let's visualize predictions on some test images to see the model in action.

In [ ]:
# Get test dataset
test_dataset_dicts = DatasetCatalog.get("electronics_test")
test_metadata = MetadataCatalog.get("electronics_test")

# Visualize predictions on random test images
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, ax in enumerate(axes):
    d = random.choice(test_dataset_dicts)
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    
    v = Visualizer(img[:, :, ::-1],
                   metadata=test_metadata,
                   scale=0.5,
                   instance_mode=ColorMode.IMAGE)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    ax.imshow(out.get_image())
    ax.axis('off')
    num_detections = len(outputs["instances"])
    ax.set_title(f"Test Sample {i+1}\n({num_detections} detections)")

plt.tight_layout()
plt.savefig('../outputs/predictions/test_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Test predictions saved to outputs/predictions/test_predictions.png")

## 8. Before/After Comparison

Compare the pre-trained COCO model (before fine-tuning) vs. our fine-tuned model.

In [ ]:
# Create predictor with pre-trained COCO weights (before fine-tuning)
cfg_before = get_cfg()
cfg_before.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg_before.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg_before.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor_before = DefaultPredictor(cfg_before)

# Pick a test image
test_img_dict = random.choice(test_dataset_dicts)
test_img = cv2.imread(test_img_dict["file_name"])

# Before: Pre-trained COCO model
outputs_before = predictor_before(test_img)
v_before = Visualizer(test_img[:, :, ::-1],
                      metadata=MetadataCatalog.get(cfg_before.DATASETS.TRAIN[0]),
                      scale=0.8)
out_before = v_before.draw_instance_predictions(outputs_before["instances"].to("cpu"))

# After: Fine-tuned model
outputs_after = predictor(test_img)
v_after = Visualizer(test_img[:, :, ::-1],
                     metadata=test_metadata,
                     scale=0.8,
                     instance_mode=ColorMode.IMAGE)
out_after = v_after.draw_instance_predictions(outputs_after["instances"].to("cpu"))

# Display side-by-side
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(out_before.get_image())
axes[0].set_title(f"BEFORE: Pre-trained COCO Model\n({len(outputs_before['instances'])} detections)", fontsize=14)
axes[0].axis('off')

axes[1].imshow(out_after.get_image())
axes[1].set_title(f"AFTER: Fine-tuned Model\n({len(outputs_after['instances'])} detections)", fontsize=14)
axes[1].axis('off')

plt.tight_layout()
plt.savefig('../outputs/predictions/before_after_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Before/after comparison saved to outputs/predictions/before_after_comparison.png")
print("\n📊 This visualization shows the improvement from fine-tuning!")

## 9. Summary

Training complete! You now have:

1. **Fine-tuned model** saved in `outputs/models/model_final.pth`
2. **Visualizations** showing:
   - Training samples
   - Test predictions
   - Before/after comparison
3. **Evaluation metrics** on validation set

### Next Steps for Production:
1. **Increase training iterations** to 2000-3000 for better results
2. **Tune hyperparameters** (learning rate, batch size, etc.)
3. **Use the inference script** (`scripts/predict.py`) for batch pre-labeling
4. **Integrate with CVAT or FiftyOne** for human review workflow

For batch pre-labeling, run:
```bash
python scripts/predict.py --input /path/to/new/images --output /path/to/predictions
```

In [ ]:
print("\n" + "="*50)
print("🎉 DEMO READY FOR TOMORROW!")
print("="*50)
print(f"\nModel location: {cfg.OUTPUT_DIR}/model_final.pth")
print(f"Visualizations: ../outputs/predictions/")
print(f"\nTotal training iterations: {cfg.SOLVER.MAX_ITER}")
print(f"Number of classes: {cfg.MODEL.ROI_HEADS.NUM_CLASSES}")
print("\nReady to demonstrate pre-labeling workflow!")